# Diffusion Models

Dans ce cours, nous allons présenter les modèles de diffusion (*diffusion models*) qui ont été introduit dans le papier [Denoising Diffusion Probabilistic Models](https://arxiv.org/pdf/2006.11239) en 2020. Depuis quelques années, ce sont ces modèles qui sont le plus utilisés pour la génération d'images (et de loin). Ils sont très performants et facilement guidables mais ils ont le défaut d'être très lent.   
Ce cours s'inspire du [CVPR 2022 Tutorial](https://cvpr2022-tutorial-diffusion-models.github.io/) et du [blogpost](https://medium.com/@gitau_am/a-friendly-introduction-to-denoising-diffusion-probabilistic-models-cc76b8abef25). Les figures utilisées dans ce notebook sont également extraites de ces deux sources.

## Comment ça marche ? 

Les *diffusion models* sont constitués de deux étapes principales : une étape de d'ajout de bruit appelée *diffusion process* et une étape pour enlever le bruit appelée *reverse process* ou *denoising process*. Ces deux étapes se font de manière itérative, c'est à dire que l'on ajoute du bruit petit à petit et qu'on l'enlève petit à petit également.

### Première étape : diffusion process

Le première étape d'un modèle de diffusion consiste à prendre une image d'un dataset quelconque. Ce dataset est representé par une distribution de probabilité très complexe. Le processus de *diffusion* consiste à ajouter un bruit gaussien sur l'image de manière itérative pour réduire petit à petit sa complexité jusqu'à ce que l'on obtienne une simple distribution gaussienne.   
La figure suivante présente le *diffusion process* : 

<img src="images/diffusion_ddpm.png" alt="diffusion_ddpm" width="800"/>

Et on peut voir la modification de la distribution de cette manière (distribution de moins en moins complexe) : 

<img src="images/distrib_ddpm.png" alt="distrib_ddpm" width="500"/>

Le *diffusion process* consiste en fait à détruire de manière itérative la structure de la donnée d'entrée (souvent une image).

Le *diffusion process* se fait en plusieurs étapes appelées *diffusion step*. Chaque étape va ajouter une quantité prédeterminée de bruit gaussien à l'image et plus il y a d'étapes, moins on va ajouter de bruit à chaque étape. En pratique, plus il y a d'étapes, plus le modèle est stable et les images produites de qualité mais plus on va avoir besoin de temps de calcul. Souvent, on va choisir un grand nombre d'étapes (1000 dans le papier original).

### Deuxième étape : reverse process

Bon c'est bien gentil mais à quoi ça sert de bruiter une image ?   
En fait, le *diffusion process* va permettre de générer des données d'entraînement pour le *reverse process*.   
L'idée du *reverse process* va être d'apprendre à passer de la distribution gaussienne (que l'on a construite avec la *diffusion*) aux images de base. Donc on veut récuperer notre image à partir du bruit gaussien de la dernière étape de notre *diffusion process*. De cette manière, on pourra aussi générer des images nouvelles à partir d'un *sample* de la distribution gaussienne. 

**Note** : On peut noter une certain similarité avec les *normalizing flow* ou les *variational autoencoders*

<img src="images/reverse_ddpm.png" alt="reverse_ddpm" width="800"/>

Pour chaque étape de *denoising*, on va utiliser un réseau de neurones qui prend en entrée l'image à l'étape $t$ et l'étape de *diffusion* $t$ et qui a pour objectif de prédire le bruit gaussien ($\mu$ et $\sigma²$) qui a été ajouté à l'image lors de l'étape $t-1 \Rightarrow t$.

**Note** : Le fait de prédire le bruit permet en fait de prédire l'image à l'étape $t-1$.

**Note 2** : Le réseau utilisé est le même à chaque étape, il n'y a pas un réseau différent par étape de diffusion.

Le modèle utilisé est généralement un modèle de type U-Net. Pour en savoir plus sur l'architecture U-Net, vous pouvez vous référer au [cours 3 sur les réseaux convolutifs](../03_RéseauConvolutifs/06_ApplicationSegmentation.ipynb). En pratique, pour les modèles très puissants (stable diffusion), une variante du U-Net incorporant l'architecture des transformers est utilisée.

<img src="images/unet_ddpm.png" alt="unet_ddpm" width="800"/>


### Les diffusion models sont des hierarchical VAE ?

Comme dit précedemment, on peut voir les *diffusion models* comme des VAE ou des *normalizing flows*.   
Regardons un peu l'analogie possible avec un VAE et plus précisement un *hierarchical* VAE.   
Un *hierarchical* VAE est un VAE qui a plusieurs étapes de génération (*decoder*) d'images. 

Pour notre *diffusion model*, le *diffusion process* correspondrait à l'*encoder* du VAE tandis que le *reverse process* correspondrait aux multiples étapes hierarchique du VAE.  
En pratique, il y a quand même quelques différences notables : 
- L'*encoder* est fixé (non entraînable) pour le diffusion model, il s'agit d'un ajout de bruit.
- L'espace latent a la même dimension que l'image d'entrée (ce qui n'est pas le cas pour un VAE).
- Le modèle utilisé pour la *diffusion* est le même à chaque étape de *diffusion*. Pour un *hierarchical VAE*, on a un modèle différent à chaque étape.  

Le même genre d'analogie peut être faite avec un *normalizing flow*, je vous invite à regarder le [CVPR 2022 Tutorial](https://cvpr2022-tutorial-diffusion-models.github.io/) pour en apprendre plus.

## Problème principal des diffusion models

Comme expliqué précedemment, les processus de *diffusion* et de *denoising* vont grandement bénéficier d'un nombre d'étapes important. Chaque étape de *denoising* va nécessiter le *forward* du réseau U-Net donc si on a 1000 étapes de *diffusion*, on va devoir appeler le réseau 1000 fois et ça pour générer une seule image. 

On voit tout de suite le problème des modèles de diffusion : ils sont très très lents.

<img src="images/generative_trilemma.png" alt="generative_trilemma" width="400"/>

Figure extraite de l'[article](https://arxiv.org/pdf/2112.07804).



Les *diffusion models* sont beaucoup plus puissants que les GANs, les VAEs et les *normalizing Flows* pour la génération d'images, c'est donc ces modèles que l'on voudrait utiliser en priorité. 

Une nécessité est alors de trouver des méthodes pour accélerer ces modèles de diffusion via diverses techniques. Ce cours ne va pas entrer dans le détail sur les techniques existantes pour accélerer ces modèles mais le [CVPR 2022 Tutorial](https://cvpr2022-tutorial-diffusion-models.github.io/) couvre très bien le sujet (bien que depuis, des nouvelles techniques ont été introduites).  
A titre informatif, on peut maintenant générer des images de qualité en une dizaine d'étapes (contre 1000 auparavant).

Les modèles de diffusion sont un gros sujet de recherche en ce moment et il reste encore beaucoup de problèmes à résoudre les concernant.   
En voici quelques-uns : 
- Pourquoi les *diffusion models* sont si performant par rapport aux VAE et aux *normalizing flows* ? Et est ce qu'on devrait rediriger les efforts de recherche sur ces alternatives maintenant que l'on a appris beaucoup de choses sur la *diffusion* ?
- Est ce qu'il serait possible de passer à une seule étape pour générer des images ? 
- Est ce que l'architecture de *diffusion* peut aider pour des applications discriminative ?
- Est ce que l'architecture U-Net est vraiment le meilleur choix pour la *diffusion* ? 
- Est ce qu'on peut appliquer les *diffusion models* sur d'autres types de données (comme du texte par exemple) ?